In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from performance_shr import convert_to_float, convert_to_arcsec, convert_to_bool, rms, plot_formating

In [ ]:

filename = '../driver/alpaca.csv' 

converters_dict = {
    'Dataset': str,   
    'Time': convert_to_float,  
    'Tracking': convert_to_bool,    
    'Slewing': convert_to_bool,    
    'Gotoing': convert_to_bool,    
    'TargetRA': convert_to_float,  
    'TargetDEC': convert_to_float,  
    'AscomRA': convert_to_float,  
    'AscomDec': convert_to_float,  
    'AscomAlt': convert_to_float,  
    'AscomAz': convert_to_float,  
    'ErrorRA': convert_to_arcsec,  
    'ErrorDec': convert_to_arcsec,  
}

raw = pd.read_csv(filename,converters=converters_dict)
raw.columns = ['Log'] + raw.columns[1:].tolist()
raw['TimeStamp'] = raw.Log.str.extract(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3})') # Extract date and time using a regular expression
raw['TimeStamp'] = pd.to_datetime(raw.TimeStamp)                                        # Convert the extracted date and time to a timestamp
raw['TimeStampSec'] = (raw.TimeStamp - raw.TimeStamp[raw.Time==0].iloc[-1]).dt.total_seconds()
raw['Goto'] = raw.Log.str.extract(r'(GOTO ASCOM .*)')                                   # Extract GOTO ASCOM commands
raw['Sync'] = raw.Log.str.extract(r'(SYNC ASCOM .*)')                                   # Extract SYNC ASCOM commands
raw['TimeStamp'].ffill()
raw.set_index('TimeStamp',inplace=True)
bad = (~raw.Tracking) | (raw.Slewing) | (raw.Gotoing)
raw.loc[bad,"ErrorRA"] = pd.NA
raw.loc[bad,"ErrorDec"] = pd.NA
df=raw


In [ ]:
df.loc[:, 'RA'] = df.ErrorRA
df.loc[:, 'Dec'] = df.ErrorDec
df.loc[:, 'RAFill'] = df.ErrorRA.ffill()
df.loc[:, 'DecFill'] = df.ErrorDec.ffill()
# df.loc[:, 'RAFill'] = df.ErrorRA.fillna(method='ffill')
# df.loc[:, 'DecFill'] = df.ErrorDec.fillna(method='ffill')
df.loc[:, 'TimeMin'] = (df.Time / 60).round(1)
df.loc[:, 'RArms15s'] = df.RA.rolling('15s',center=True).apply(rms, raw=True)
df.loc[:, 'Decrms15s'] = df.Dec.rolling('15s',center=True).apply(rms, raw=True)
df.loc[:, 'RAStdev15s'] = df.RA.rolling('15s',center=True).std()
df.loc[:, 'DecStdev15s'] = df.Dec.rolling('15s',center=True).std()


In [ ]:
fig=go.Figure()
title,labels,xtitle,ytitle,bgcolor = plot_formating('Periodic Error (Dec Axis) vs Time Elapsed','Time Elapsed (seconds)','Perioidic Error (arc seconds)')
fig.update_layout(title=title, xaxis_title=xtitle, yaxis_title=ytitle, plot_bgcolor=bgcolor)
# Add trace for Dec Error
fig.add_trace(go.Scatter(x=df.TimeStampSec, y=df.Dec, name='Dec Error', mode='markers'))
# Add points for all GOTOs
fig.add_trace(go.Scatter(
    x=df[df.Goto.notna()].TimeStampSec, y=df[df.Goto.notna()].DecFill,
    name='Goto', text=df[df.Goto.notna()].Goto, mode='markers', 
    marker=dict(size=10, color='red')
))
# Add points for all Syncs
fig.add_trace(go.Scatter(
    x=df[df.Sync.notna()].TimeStampSec, y=df[df.Sync.notna()].DecFill,
    name='Sync', text=df[df.Sync.notna()].Sync, mode='markers', 
    marker=dict(size=10, color='dark green')
))
# Add trace for rms
fig.add_trace(go.Scatter(x=df.TimeStampSec, y=df.Decrms15s, name='15s rms', mode='markers'))


fig.show()

In [ ]:
fig=go.Figure()
title,labels,xtitle,ytitle,bgcolor = plot_formating('Periodic Error (RA Axis) vs Time Elapsed','Time Elapsed (seconds)','Periodic Error (arc seconds)')
fig.update_layout(title=title, xaxis_title=xtitle, yaxis_title=ytitle, plot_bgcolor=bgcolor)
# Add trace for Dec Error
fig.add_trace(go.Scatter(x=df.TimeStampSec, y=df.RA, name='Periodic Error', mode='markers'))
# Add points for all GOTOs
fig.add_trace(go.Scatter(
    x=df[df.Goto.notna()].TimeStampSec, y=df[df.Goto.notna()].RAFill,
    name='Goto', text=df[df.Goto.notna()].Goto, mode='markers', 
    marker=dict(size=10, color='red')
))
# Add points for all Syncs
fig.add_trace(go.Scatter(
    x=df[df.Sync.notna()].TimeStampSec, y=df[df.Sync.notna()].RAFill,
    name='Sync', text=df[df.Sync.notna()].Sync, mode='markers', 
    marker=dict(size=10, color='dark green')
))
# Add trace for rms
fig.add_trace(go.Scatter(x=df.TimeStampSec, y=df.RArms15s, name='15s rms', mode='markers'))
fig.show()

In [ ]:
m = df.melt(id_vars='Time', value_vars=['RA', 'Dec'], var_name='Axis', value_name='Error')
title,labels,xtitle,ytitle,bgcolor = plot_formating('Periodic Error vs Time Elapsed','Time Elapsed (seconds)','Periodic Error (arc seconds)')
fig=px.scatter(x=m.Time, y=m.Error, color=m.Axis, facet_col=m.Axis, labels=labels)
fig.update_layout(title=title, xaxis_title=xtitle, yaxis_title=ytitle, plot_bgcolor=bgcolor)
# Add points for all GOTOs
fig.add_trace(go.Scatter(
    x=df[df.Goto.notna()].TimeStampSec, y=df[df.Goto.notna()].RAFill,
    name='Goto', text=df[df.Goto.notna()].Goto, mode='markers', 
    marker=dict(size=10, color='red'),
    ) 
)
# Add points for all GOTOs
fig.add_trace(go.Scatter(
    x=df[df.Goto.notna()].TimeStampSec, y=df[df.Goto.notna()].DecFill,
    name='Goto', text=df[df.Goto.notna()].Goto, mode='markers', 
    marker=dict(size=10, color='red'),
    ), row=1, col=2
)# Add points for all GOTOs
fig.add_trace(go.Scatter(
    x=df[df.Sync.notna()].TimeStampSec, y=df[df.Sync.notna()].RAFill,
    name='Sync', text=df[df.Sync.notna()].Sync, mode='markers', 
    marker=dict(size=10, color='green'),
    ) 
)
# Add points for all GOTOs
fig.add_trace(go.Scatter(
    x=df[df.Sync.notna()].TimeStampSec, y=df[df.Sync.notna()].DecFill,
    name='Sync', text=df[df.Sync.notna()].Sync, mode='markers', 
    marker=dict(size=10, color='green'),
    ), row=1, col=2
)
fig.show()